In [4]:
file_path = 'https://raw.githubusercontent.com/ek-chris/Practice_datasets/refs/heads/main/EasyVisa%20(1).csv'

In [6]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import warnings
warnings.filterwarnings('ignore')

# Preprocessing libraries
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix

# Statistical libraries
from scipy import stats
from scipy.stats import zscore, skew

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

def load_check_data_quality(file_path):
    """
    Loads dataset, performs basic data quality check
    
    Steps:
    1. Load CSV file.
    2. check missing value.
    3. check for duplicate.
    4. Check skewness for variables identified in EDA as right-skewed.
  
    """

    # === 1. Load dataset ===
    try:
        df = pd.read_csv(file_path)
        print(f"Dataset loaded successfully: {file_path}")
        print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns\n")
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

    # === 1. Missing Values ===
    print("\n=== MISSING VALUE PERCENTAGES ===")
    missing = df.isnull().mean() * 100
    print(missing[missing > 0].sort_values(ascending=False))

    # 2. Check for duplicates
    print("\n2. Duplicate Rows:")
    duplicates = df.duplicated().sum()
    print(f"Number of duplicate rows: {duplicates}")
    if duplicates > 0:
        print(f"Percentage of duplicates: {(duplicates/len(df))*100:.2f}%")

    # 3. Check skewness for variables identified in EDA as right-skewed
        print("\n3. Skewness Analysis (EDA identified right-skewed variables):")
        skewed_vars = ['no_of_employees', 'yr_of_estab', 'prevailing_wage ']
        for var in skewed_vars:
            if var in df.columns:
                skewness = skew(df[var])
                print(f"{var}: skewness = {skewness:.3f} "
                f"({'strongly skewed' if abs(skewness) > 0.7 else 'moderately skewed' if abs(skewness) > 0.3 else 'approximately normal'})")

    # 4. Check correlation with target (EDA evidence)
    # print("\n4. Correlation with case_status (EDA Evidence):")
    # correlations = df.corr()['case_status'].sort_values(key=abs, ascending=False)
    # print("High-signal features (|correlation| > 0.2):")
    # high_signal = correlations[abs(correlations) > 0.2].drop('case_status')
    # for feature, corr in high_signal.items():
    #     print(f"  {feature}: {corr:.3f}")

    return df


Libraries imported successfully!


In [7]:
df = load_check_data_quality(file_path)

Dataset loaded successfully: https://raw.githubusercontent.com/ek-chris/Practice_datasets/refs/heads/main/EasyVisa%20(1).csv
Shape: 25480 rows × 12 columns


=== MISSING VALUE PERCENTAGES ===
Series([], dtype: float64)

2. Duplicate Rows:
Number of duplicate rows: 0


Handling outliers

In [9]:
# Outlier treatment based on EDA recommendations
print("=== OUTLIER TREATMENT (IQR-CAPPING METHOD) ===")
print("EDA recommended IQR-capping for  to preserve data points")

# Define numerical columns (excluding target)
numerical_cols =  ['no_of_employees', 'yr_of_estab', 'prevailing_wage']
if 'Loan_Status' in numerical_cols:
    numerical_cols.remove('Loan_Status')

print(f"Treating outliers in {len(numerical_cols)} numerical features...")

# Apply IQR-capping method
outliers_capped = 0
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Count outliers before capping
    outliers_before = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    
    if outliers_before > 0:
        # Cap outliers
        df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
        df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
        outliers_capped += outliers_before
        print(f"✓ {col}: Capped {outliers_before} outliers")

print(f"\nTotal outliers capped: {outliers_capped}")
print(f"Dataset shape after outlier treatment: {df.shape}")

=== OUTLIER TREATMENT (IQR-CAPPING METHOD) ===
EDA recommended IQR-capping for  to preserve data points
Treating outliers in 3 numerical features...

Total outliers capped: 0
Dataset shape after outlier treatment: (25480, 12)


 Encoding(like label encoding and one-hot encoding)**

In [19]:
import pandas as pd

def preprocess_employee_data(df):
    """
    Preprocess the employee dataset by encoding categorical variables.
    - Binary categorical columns: Label encoded using custom mappings
    - Multi-category columns: One-hot encoded
    """

    # Label Encoding (binary columns)
    label_map = {
        'has_job_experience': {'Y': 1, 'N': 0},
        'requires_job_training': {'Y': 1, 'N': 0},
        'full_time_position': {'Y': 1, 'N': 0},
        'case_status': {'Certified': 1, 'Denied': 0}
    }

    for col, mapping in label_map.items():
        if col in df.columns:
            df[col] = df[col].map(mapping)

    # One-hot Encoding (multi-category columns)
    onehot_cols = [
        'continent',
        'education_of_employee',
        'region_of_employment',
        'unit_of_wage'
    ]

    df = pd.get_dummies(df, columns=onehot_cols, drop_first=False, dtype=int)

    print("\nPreprocessing complete.")
    return df


In [ ]:
df_processed = preprocess_employee_data(df)

print(df_processed.head())


Preprocessing complete.
  case_id  has_job_experience  requires_job_training  no_of_employees  \
0  EZYV01                   0                      0           7227.0   
1  EZYV02                   1                      0           2412.0   
2  EZYV03                   0                      1           7227.0   
3  EZYV04                   0                      0             98.0   
4  EZYV05                   1                      0           1082.0   

   yr_of_estab  prevailing_wage  full_time_position  case_status  \
0       2007.0         592.2029                   1            0   
1       2002.0       83425.6500                   1            1   
2       2008.0      122996.8600                   1            0   
3       1932.5       83434.0300                   1            0   
4       2005.0      149907.3900                   1            1   

   continent_Africa  continent_Asia  ...  education_of_employee_Master's  \
0                 0               1  ...           